# Stations database
The objective of stations database is to create a unique set of station-access combinations that is consistent for all years of observations. Some of the problems with the stations and access are: some stations have changed names, new access points are added or dropped, some stations have two unique ids (mostly to differentiate feeder entrances). 

When a data base of unique stations-access combanations is created, I will manually create a map that matches the station-access combination to a unique and consistent station ID. Additionally, stations will have walk, feeder and intercity access differentiated (Some stations have different IDs for feeder routes, but other stations have the same ID, but the difference can be made at the access point). This differentiation needs to be validated by Transmilenio because it is not always easy to infer it base on the access name. 

We dropped all DUAL stations (they only account for ~3.5% of the total transactions in the system)

There are two cleaning procedures bacause files have different formats: 
1. June 2015 to June 2018 (we previously created an "indent" column to differentiate the zone (indent 0), the station (indent 1), the access (indent 3) and the time (indent 4).  
2. July 2018 to Present

In [1]:
import pandas as pd 
import numpy as np
import xlrd
import csv

import os
from os import listdir
import functools
import glob
import datetime
import matplotlib.pyplot as plt

## Reading files - raw data

In [3]:
## Read all files 
PATH = '../data/transactions'
all_files = []
for root,dirs,files in os.walk(PATH):
    for file in files:
        all_files.append(os.path.join(root, file))
        
## Files June 2015 to June 2017
files_2015 = list(filter(lambda f: f.endswith('.csv'), all_files))

#Files July 2017 to April 2021
years = '2017|2018|2019|2020|2021'
months = 'Enero-2017|Febrero-2017|Marzo-2017|Abril-2017|Mayo-2017|Junio-2017|~'
files_2017 = pd.Series(list(filter(lambda f: f.endswith('.xlsx'), all_files)))
files_2017 = files_2017[files_2017.str.contains(years)]
files_2017 = files_2017[~files_2017.str.contains(months)]

files = files_2015 + list(files_2017)
files = [files[0]] + files[3:] #Dropping Jun and July 

In [5]:
%%time 
raw_month = []
sheet = 'Validaciones Consolidado'
sheet = 'Validaciones Tullave'
for file in files:
    if file.endswith('.csv'):
        raw_month.append(pd.read_csv(file, header = 6))
    elif file.endswith('.xlsx'):
        condition = '2017|2018|Enero 2019|Febrero 2019'
        consolidado = pd.Series(file).str.contains(condition)[0]
        if consolidado:
            sheet = 'Validaciones Consolidado'
        else:
            sheet = 'Validaciones Tullave'

        raw_month.append(pd.read_excel(file, header = 6,sheet_name=sheet))

CPU times: user 22min 22s, sys: 10.9 s, total: 22min 33s
Wall time: 22min 46s


In [6]:
# Import list of clean station that I had from previous reserach. 
# This stations only contain information of 2020. Data from previous 
# year is not consistent with this data. 

# This file can be found in https://github.com/jdcaicedo251/transmilenio-covid
stations_path = '../../../01_tm_covid/transmilenio-covid/data/input/stations.csv'
old_stations = pd.read_csv(stations_path)

## Functions 

In [50]:
#Classify zones in BRT or DUAL, depending to where they belong. 
stations_classification_dict = {'(11) Zona K Calle 26':'BRT', '(12) Zona L Carrera 10':'BRT',
       '(122) DUAL_M84 - C84':'DUAL', '(125) DUAL_K86 - M86':'DUAL', '(130) M82-L82':'DUAL',
       '(31) Zona F Av. Américas':'BRT', '(32) Zona C Av. Suba':'BRT','(35) Zona D Calle 80':'BRT',
       '(36) Zona A Caracas':'BRT', '(37) Zona J Eje Ambiental':'BRT', '(121) DUAL_M80_L80':'DUAL', 
       '(125) M86-K86':'DUAL', '(201) D81-H81':'DUAL', '(30) Zona G NQS Sur':'BRT', 
       '(33) Zona B AutoNorte':'BRT','(34) Zona H Caracas Sur':'BRT', '(38) Zona E NQS Central':'BRT',
       '(40) Zona T Ciudad Bolívar':'BRT','Total general':'Other', '(35101) /':'DUAL','(135) M81 - H81':'DUAL',
       '(30) Zona G NQS':'BRT', '(33) Zona B Norte':'BRT', '(34) Zona H Tunal': 'BRT', 'Dual D81-H81':'DUAL', 
       '(135) M81 - H81':'DUAL', '(135) M81-H81':'DUAL', '(34470)':'DUAL', '(135) H83-M83':'DUAL', 
       '(35882) /':'DUAL', '(23334) /':'DUAL', '(23757) /':'DUAL', '(35132) /':'DUAL' ,'(41444) /':'DUAL', 
       '(84)':'DUAL', '(07008) NQS - Calle 30 Sur':'BRT', '(39) Zona F Calle 13':'BRT', 
       '(252) D81-M81':'DUAL', '(253) H83-M83':'DUAL', '(201) Dual D81-H81':'DUAL', '(210) M80 - L80':'DUAL'}

In [70]:
def stations_data_base(df, classification, drop_brt = True):
    ''' Obtains unique combinations Station-Access. 
    Parameters: 
    ------------
    - df: raw transactions data 
    - classification: dict. Classifies zones in BRT and DUAL. Key: zone, value:'BRT'/'DUAL'/'Other'
                        -BRT: Zone is a BRT zone 
                        -DUAL: Zone is a Dual route
                        - Other: Zone is "Total general" (aggreage measure in the raw data)

    ------------
    Returns: transactions every 15 mins. Index is timestamp, columns are stations
    '''
    ### asserts 
    # classification stations are in df 
    # values in classifications are only DUAL or BRT or Other
    
    #For June 2015 to June 2018
    #note: Column indent was created by us manually in the raw excel file
    # to separate zone, station, access, and time. 
    if 'indent' in df.columns:
    
        time_indent = 3
        indent_level = {'zone':0, 'station':1, 'access':2}

        new_cols = ['zone', 'station','access']
        for key, value in indent_level.items(): 
            df[key] = df['indent']
            df[key] = df[key].where(df[key]!=value,df['Etiquetas de fila']).astype(str)
            df[key] = df[key].replace(to_replace = '^\\d', value =  np.nan, regex = True)\
                                .fillna(method = 'ffill')
        #Keep only time indent 
        df = df[df.indent == time_indent].drop(columns = ['indent','Total general'])
        df.rename(columns = {'Etiquetas de fila':'time'}, inplace = True)
     
    # For July 2018 to Present 
    else:
        
        renanme_cols = {'Línea':'zone','Estación':'station', 'Acceso de Estación':'access'}
        df = df[['Línea','Estación', 'Acceso de Estación']].rename(columns = renanme_cols)
        df.dropna(inplace = True)
        
    #Drop DUAL stations 
    ###########
    if drop_brt:
        
        df['type'] = df['zone'].replace(classification)
    
        #Make sure all zones have a classification
        non_existing_zones = df[~df.type.str.contains('BRT|DUAL|Other')]['zone']
        if non_existing_zones.shape[0] > 0:
            print(non_existing_zones.unique())

        df = df[df.type == 'BRT'].drop(columns = 'type')
        
    dates = df.columns[df.columns.str.contains(pat="\d", regex = True)]
    df['transactions'] = df[dates].replace(',','', regex=True).fillna(0).astype(int).sum(axis = 1)
        
    #Unique station-access pair 
    ##################
    df['station_access'] = df['station'] + '-' + df['access']
    df = df[['zone', 'station','access','station_access', 'transactions']]#.drop_duplicates(subset = 'station_access')
    
    return df

## Understanding stations 

In [71]:
df = raw_month[0]
nov = stations_data_base(df,stations_classification_dict, False)

stations = nov[['zone','station']]
stations = stations.drop_duplicates()

In [72]:
unique_zones = stations.zone.nunique()
unique_stations = stations.station.nunique()
print ('There are', unique_zones, 'unique zones')
print ('There are', unique_stations, 'unique stations')

There are 18 unique zones
There are 186 unique stations


In [73]:
#Zones
counter = 1
for zone in stations.zone.unique():
    print('ZONE:', counter)
    df = stations[stations.zone == zone]
    print ('Number of stations',df.shape[0])
    print (df)
    print ('')
    counter += 1

ZONE: 1
Number of stations 14
                      zone                         station
3     (11) Zona K Calle 26         (06000) Portal Eldorado
327   (11) Zona K Calle 26                 (06001) Modelia
405   (11) Zona K Calle 26               (06002) Normandía
486   (11) Zona K Calle 26               (06100) Av. Rojas
565   (11) Zona K Calle 26               (06101) El Tiempo
645   (11) Zona K Calle 26        (06102) Salitre El Greco
728   (11) Zona K Calle 26                     (06103) CAN
809   (11) Zona K Calle 26             (06104) Gobernación
890   (11) Zona K Calle 26          (06105) Quinta Paredes
968   (11) Zona K Calle 26               (06106) Corferias
1050  (11) Zona K Calle 26    (06107) Ciudad Universitaria
1210  (11) Zona K Calle 26  (06108) Plaza de la Democracia
1289  (11) Zona K Calle 26          (06109) Centro Memoria
1449  (11) Zona K Calle 26           (06111) Universidades

ZONE: 2
Number of stations 11
                        zone                      stat

In [74]:
# stations_classification_dict = {'(11) Zona K Calle 26':'BRT', '(12) Zona L Carrera 10':'BRT',
#        '(122) DUAL_M84 - C84':'DUAL', '(125) DUAL_K86 - M86':'DUAL', '(130) M82-L82':'DUAL',
#        '(31) Zona F Av. Américas':'BRT', '(32) Zona C Av. Suba':'BRT',
#        '(35) Zona D Calle 80':'BRT', '(36) Zona A Caracas':'BRT',
#        '(37) Zona J Eje Ambiental':'BRT', '(121) DUAL_M80_L80':'DUAL', '(125) M86-K86':'DUAL',
#        '(201) D81-H81':'DUAL', '(30) Zona G NQS Sur':'BRT', '(33) Zona B AutoNorte':'BRT',
#        '(34) Zona H Caracas Sur':'BRT', '(38) Zona E NQS Central':'BRT',
#        '(39) Zona F Calle 13':'BRT'}

nov['type'] = nov['zone'].replace(stations_classification_dict)
summary_brt_dual = nov.groupby('type').agg({'transactions':'sum'})
summary_brt_dual['total'] = summary_brt_dual['transactions'].sum()
summary_brt_dual['pct'] = summary_brt_dual['transactions']/summary_brt_dual['total']

print ('% of BRT Transactions: {:.2f}'.format(summary_brt_dual['pct'][0]*100))
print ('% of DUAL Transactions: {:.2f}'.format(summary_brt_dual['pct'][1]*100))

% of BRT Transactions: 96.56
% of DUAL Transactions: 3.44


## Create final stations database

In [75]:
raw_stations = []
for month in raw_month:
    df = stations_data_base(month, stations_classification_dict, True)
    raw_stations.append(df)

### Some facts about this dataset 

In [76]:
stations_database = pd.concat(raw_stations, axis = 0).drop_duplicates(subset = 'station_access') 
# I should have drop duplicates in the colum "station_access". I had dropthem manually. 
print ('There are {} unique station-access combination in the observed data'.format(stations_database.shape[0]))

There are 991 unique station-access combination in the observed data


In [77]:
# Unique station-access combination 
old_stations['station_access'] = old_stations['nombreestacion'] + '-' + old_stations['nombreaccesoestacion']

#Merge to check for matches 
merge = old_stations.merge(stations_database, how = 'outer', on = 'station_access')

missing_combination = merge[merge.nombrelinea.isnull()].shape[0]
stations_not_found = merge[merge.zone.isnull()].shape[0]
print ('There are {} additional combinations in our dataset'.format(missing_combination))
print ('There are {} additional combinations in old dataset '.format(stations_not_found))

There are 420 additional combinations in our dataset
There are 23 additional combinations in old dataset 


In [78]:
merge.to_csv('../data/stations_database_v2.csv', index = False)

At this point, I manually checked that all addtional stations match to an existing station. I have also:
- Drop all duplicates for station_access combination. There are some station for which the zone had a different name, therefore the observation was not dropped before. This resulted in duplicated transaction when matching stations and access. 
- Ricaurte and Avenida Jimenez belong to two zones each. For the purpose of prediction, I have joined the demand for each station. 